<a href="https://colab.research.google.com/github/sergope/tfm/blob/main/normalize_data_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Conectar el notebook con la cuenta de gdrive
from google.colab import drive
drive.mount('/content/drive/', force_remount=False)

BASE_FOLDER = 'drive/My Drive/TFM/resources/'

# 4/5QEd0fKmDZNAa31VQJkz2jbunYtoTkIO7_mXkg-i3fTokiPDB8cmJxc

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# DATE FUNCTIONS
def date_normalized_row(row):
  mes = str(row['MES'])
  if (row['MES'] < 10):
    mes = '0' + str(row['MES'])

  dia = str(row['DIA'])
  if (row['DIA'] < 10):
    dia = '0' + str(row['DIA'])

  date_normalized = str(row['ANO']) + '-' + mes + '-' + dia
  return date_normalized

def date_normalized(table):
  table['ds'] = table.apply(lambda row: date_normalized_row(row), axis=1)
  return table

In [ ]:
# MAGNITUDE FUNCTIONS
def filter_by_magnitude(table, magnitude):
  return table.loc[(table['MAGNITUD'] == magnitude)]

def extract_daily_max_magnitude(table):
  hours_range = ['H01','H02','H03','H04','H05','H06','H07','H08','H09','H10','H11','H12','H13','H14','H15','H16','H17','H18','H19','H20','H21','H22','H23','H24',]
  table['y'] = table.loc[:, hours_range].max(axis=1)
  return table


In [ ]:
# CLEAN TABLE FUNCTIONS
def filter_columns(table, columns_list):
  return table.filter(columns_list)

In [ ]:
def prophet_data_normalized_one_magnitude(file_name, magnitude):
  #1- LEO CSV HISTORICO DE UNA ESTACIÓN
  table = pd.read_table(BASE_FOLDER+'processed_data/' + file_name + '.csv', sep=';', engine='python')
  #2- FORMATEO FECHA
  table = date_normalized(table)
  #3- FILTRO POR MAGNITUD
  table = filter_by_magnitude(table, magnitude)
  #4- MAXIMO DIARIO
  table = extract_daily_max_magnitude(table)
  #4.1- COMPROBAR QUE LAS MEDIDAS ESTÁN VERIFICADAS
  #5- ELIMINAR COLUMNAS INNECESARIAS
  table = filter_columns(table, ['ds', 'y'])
  #6- ELIMINAR NAN
  table.dropna(inplace=True)
  #7- ACTUALIAR ÍNDICE
  table.reset_index(drop=True, inplace=True)
  #8- DEVOLVEMOS LA TABLA
  return table


In [ ]:
# Normalize aemet data
def normalize_aemet_data(file_name, columns_to_filter):
  table = pd.read_table(BASE_FOLDER+'processed_data/' + file_name + '.csv', sep=';', engine='python')
  table.rename(columns={"FECHA": "ds"}, inplace=True)
  columns_with_date = columns_to_filter.copy()
  columns_with_date.append('ds')
  table = filter_columns(table, columns_with_date)
  table.drop_duplicates(subset ="ds", keep = 'last', inplace = True) 
  for column in table.columns:
    table = table[table[column] != 'Ip']
    table[column] = table[column].str.replace(',', '.')

  return table